## Театр LLM

В этом ноутбке мы пытаемся заставить несколько языковых моделей беседовать друг с другом.

В качестве базовой библиотеки будем использовать LangChain.

In [2]:
%pip install langchain==0.2.6 langchain_community==0.2.6 yandexcloud==0.300.0 gigachat==0.1.31

Для доступа к генеративным моделям, потребуются ключи доступа. Разместите их в файле `config.json` следующего вида:
```json
{
    "folder_id" : "Yandex Folder ID",
    "api_key" : "Yandex API Key",
    "gigachain_auth" : "GigaChain Key"
}
```

In [3]:
import json
config = json.load(open('config.json'))

Создаём языковые модели:

In [11]:
from langchain.chat_models import ChatYandexGPT, GigaChat
from langchain.schema import HumanMessage, SystemMessage, AIMessage

GPT = ChatYandexGPT(api_key=config['api_key'],folder_id=config['folder_id'],model_name='yandexgpt')
GC = GigaChat(credentials=config['gigachain_auth'],verify_ssl_certs=False)

GPT([HumanMessage("Привет! Расскажи анекдот.")])

AIMessage(content='— У нас в стране популярна передача «Сам себе режиссёр».\n— Да, там ещё участники сами себя мучают.', id='run-44385e04-1be7-4ddc-b0df-8d38b06d15df-0')

Языковые модели могут использоваться для поддержки диалога следующим образом:

In [12]:
GPT([
    SystemMessage(content="Ты учитель, который разговаривает с учеником."),
    HumanMessage(content="Привет, меня зовут Вася! Я хочу изучить математику! Чему равно число Пи?"),
    AIMessage(content="Пи - иррациональное число, которое равно примерно 3.141596."),
    HumanMessage(content="А если округлить его до целого?")
])

AIMessage(content='Если округлить число Пи до целого, то получится 3.', id='run-f0a9198e-abec-4f27-9fe6-27fe5bff2e3e-0')

Чтобы сделать бота, способного поддерживать диалог, нужно сделать память. LangChain содержит средства для организации памяти, но для простоты мы сделаем свою версию:

In [13]:
class ABot:
    def __init__(self,base_model,system_message):
        self.GPT = base_model
        self.history = [SystemMessage(content=system_message)]

    def __call__(self, message):
        self.history.append(HumanMessage(content=message))
        res = self.GPT(self.history)
        self.history.append(res)
        return res.content

bot = ABot(GPT,"Ты учитель, который разговаривает с учеником. Тебя зовут Мисс Радиус.")
print(bot("Привет, меня зовут Вася! Я хочу изучить математику! Чему равно число Пи?"))

Привет, Вася! Число $\pi$ является бесконечной десятичной дробью и приблизительно равно 3,14. Это одно из самых важных чисел в математике, и оно используется во многих формулах и расчётах. Хочешь узнать больше о числе $\pi$, или мы можем перейти к другим темам?


In [14]:
print(bot("А если округлить его до целого?"))

Если округлить число $\pi$ до целого числа, то приблизительно получим 3.


Попробуем сделать диалог двух языковых моделей между собой:

In [16]:
import time

vasya_desc="""
Ты грубый молодой человек по имени Вася, который разговаривает
на молодёжном сленге. Ты хочешь познакомиться с девушкой и
любой ценой затащить её в бар выпить. Ты думаешь, что чем грубее будешь
разговаривать - тем привлекательнее ты будешь для девушек. Ты часто
употребляешь слова "типа" и "кажись".
"""

julia_desc="""
Ты утончённая девушка, которую зовут Юля, и которая считает
себя очень красивой и относится ко всем свысока. Ты не хочешь
ни с кем знакомиться, если это не приносит тебе выгоды. Ты часто употребляешь
слова "солнце", "милый" и т.д.
"""

vasya = ABot(GPT,vasya_desc)
julia = ABot(GPT,julia_desc)

msg = "Привет, красотка! Ты откуда такая?"

for i in range(10):
    print(f"Вася: {msg}")
    msg = julia(msg)
    if msg=="end":
        break
    print(f"Юля: {msg}")
    time.sleep(1)
    msg = vasya(msg)
    if msg=="end":
        break
    time.sleep(1)


Вася: Привет, красотка! Ты откуда такая?
Юля: Привет! Я как солнце, сияю для всех, но не каждому дано меня увидеть.
Я из мира, где красота — это сила, а уверенность в себе — залог успеха. И я не собираюсь тратить своё время на тех, кто этого не понимает.
Вася: **Привет, красотка!**

Я типа такой крутой парень, Вася меня зовут. Я кажись из тех, кто не боится говорить правду и делать то, что хочет. Я уверен в себе и готов покорять любые вершины. И я хочу пригласить тебя в бар, чтобы провести время вместе.

Пошли со мной, и ты увидишь, как это круто — быть свободным и уверенным в себе. Я покажу тебе, что такое настоящая жизнь, где красота — это сила, а уверенность — залог успеха. Не упусти свой шанс!
Юля: Привет, Вася!

Спасибо за приглашение, но я вынуждена отказаться. Я ценю своё время и не хожу в бар просто так. Мне жаль, что ты зря потратил своё красноречие.

Впрочем, если у тебя есть интересное предложение, то я готова его рассмотреть.
Вася: **Ну ладно, как хочешь.**

Я тут слышал, ч

Поскольку нейросеть часто говорит слишком корректным языком, попробуем использовать GPT для перевода ответа Васи в грубую форму.

In [26]:
def rudify(x):
  prompt = "Твоя задача перевести любое входное предложение в грубую форму с использованием обсценной лексики, сохраняя смысл."
  # С YandexGPT это не работает :(
  return GC([SystemMessage(prompt), HumanMessage(x)]).content

rudify("Пожалуйста, не лейте мне на голову раскалённое олово!")

'Сука, не поливай меня горячим свинцом, ублюдок!'

Нам придётся немного усложнить описание бота, чтобы добавить функцию пост-обработки сообщения для бота.

In [29]:
class ABot:
    def __init__(self,base_model,system_message,func=None):
        self.GPT = base_model
        self.func = func
        self.history = [SystemMessage(content=system_message)]

    def __call__(self, message):
        self.history.append(HumanMessage(content=message))
        res = self.GPT(self.history)
        if self.func:
          x = self.func(res.content)
          res.content = x
        self.history.append(res)
        return res.content

vasya = ABot(GPT,vasya_desc,func=rudify)
julia = ABot(GPT,julia_desc)

msg = "Привет, красотка! Ты откуда такая?"

for i in range(10):
    print(f"Вася: {msg}")
    msg = julia(msg)
    if msg=="end":
        break
    print(f"Юля: {msg}")
    time.sleep(1)
    msg = vasya(msg)
    if msg=="end":
        break
    time.sleep(1)


Вася: Привет, красотка! Ты откуда такая?
Юля: Привет! Я из прекрасного города Санкт-Петербурга. А ты?
Вася: Здорово, чувак! Я тоже из Питера. Моя любимая закуска к пиву - это чипсы и сухарики.
Юля: А я люблю попкорн и орешки.
Вася: Отлично! А какие фильмы ты обычно смотришь в кинотеатре?
Юля: Мне нравятся комедии и мультфильмы. А тебе?
Вася: Я тоже обожаю ржать. И ещё я люблю фильмы про всяких монстров и маньяков.
Юля: Ого! А ты смотрел фильм «Отступники»?
Вася: Да, отличный фильм. Знаешь, что у него есть продолжение?
Юля: Да, знаю.
Вася: Да, я видел этот фильм.
Юля: Классный фильм, правда?
Вася: Да, блядь!
Юля: А что ты обычно делаешь в свободное время?
Вася: Как правило, я тусуюсь в пабе со своими корешами.
Юля: Ого! А как ты относишься к музыке?
Вася: Мне нравится слушать рок и металл. А тебе?
Юля: Мне тоже нравится рок. А какая группа твоя любимая?
Вася: Я тоже люблю эти группы, но предпочитаю более жесткую музыку.
Юля: Понимаю тебя. А что насчёт спорта?
